# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
0.983033,"""70127697861485…","""11875290780184…","""../../../../da…","""15814376529087…","""15089540292191…",false
0.144221,"""32051145628309…","""10079463819776…","""../../../../da…","""15814376529087…","""16003829073198…",true
0.679745,"""84873740928728…","""58489759412240…","""../../../../da…","""46706785537079…","""15089540292191…",false
1.0,"""18107740119210…","""10079463819776…","""../../../../da…","""64221670025601…","""15089540292191…",false
0.718982,"""17097451940834…","""11875290780184…","""../../../../da…","""64221670025601…","""90706395051551…",true
0.0,"""60017070315459…","""58489759412240…","""../../../../da…","""17249587739591…",null,false
0.02333,"""41187175719563…","""58489759412240…","""../../../../da…","""64221670025601…","""90706395051551…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_City': 'PREDICTOR_1',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
0.983033,"""11720912151873…","""11387096834964…","""../../../../da…","""FirstMortgage3…","""15783680313283…","""Rejected"""
0.144221,"""23312617431889…","""15212857817096…","""../../../../da…","""FirstMortgage3…","""57325847939000…","""Accepted"""
0.679745,"""18736534409423…","""11056236983149…","""../../../../da…","""MoneyMarketSav…","""15783680313283…","""Rejected"""
1.0,"""13971770371639…","""15212857817096…","""../../../../da…","""BasicChecking""","""15783680313283…","""Rejected"""
0.718982,"""96971198403684…","""11387096834964…","""../../../../da…","""BasicChecking""","""15752138599477…","""Accepted"""
0.0,"""44467369621138…","""11056236983149…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""90805897397771…","""11056236983149…","""../../../../da…","""BasicChecking""","""15752138599477…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
0.983033,"""10004759209059…","""80999040033044…","""../../../../da…","""FirstMortgage3…","""25180006961239…","""Rejected"""
0.144221,"""33991059680294…","""11942209409214…","""../../../../da…","""FirstMortgage3…","""17355899435316…","""Accepted"""
0.679745,"""16800283572800…","""10103006062107…","""../../../../da…","""MoneyMarketSav…","""25180006961239…","""Rejected"""
1.0,"""12870768821964…","""11942209409214…","""../../../../da…","""BasicChecking""","""25180006961239…","""Rejected"""
0.718982,"""75056836512247…","""80999040033044…","""../../../../da…","""BasicChecking""","""17010021060988…","""Accepted"""
0.0,"""46032167478123…","""10103006062107…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""15814017989778…","""10103006062107…","""../../../../da…","""BasicChecking""","""17010021060988…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_City=PREDICTOR_1
Customer_MaritalStatus=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
0.983033,"""41231511715996…","""56637952753377…","""FirstMortgage3…","""10025102307010…","""Rejected"""
0.144221,"""38795571429605…","""56303790491041…","""FirstMortgage3…","""41015403264709…","""Accepted"""
0.679745,"""12778572064331…","""12065110859377…","""MoneyMarketSav…","""10025102307010…","""Rejected"""
1.0,"""74232927240192…","""56303790491041…","""BasicChecking""","""10025102307010…","""Rejected"""
0.718982,"""34736019899038…","""56637952753377…","""BasicChecking""","""76046502193108…","""Accepted"""
0.0,"""14230525213142…","""12065110859377…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""14325284538948…","""12065110859377…","""BasicChecking""","""76046502193108…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
0.983033,"""c73023dda51de1…","""af486bc1815c72…","""../../../../da…","""FirstMortgage3…","""2513d0e24d8d02…","""Rejected"""
0.144221,"""5414b197fb87b3…","""3cdf7b1ae1342e…","""../../../../da…","""FirstMortgage3…","""8b55bf62fc2ecd…","""Accepted"""
0.679745,"""892f5d61a9215e…","""806ef34eb8237f…","""../../../../da…","""MoneyMarketSav…","""2513d0e24d8d02…","""Rejected"""
1.0,"""c3aa807d9e927a…","""3cdf7b1ae1342e…","""../../../../da…","""BasicChecking""","""2513d0e24d8d02…","""Rejected"""
0.718982,"""c2d492f9a05073…","""af486bc1815c72…","""../../../../da…","""BasicChecking""","""1cd082ecbd5dd9…","""Accepted"""
0.0,"""b21059221605c4…","""806ef34eb8237f…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""80cbe489dcecd4…","""806ef34eb8237f…","""../../../../da…","""BasicChecking""","""1cd082ecbd5dd9…","""Rejected"""
